In [1]:
import moha
import numpy as np
from pyscf import gto, scf, fci


In [2]:
def RanGo(N):
    h2 = np.zeros((2*N, 2*N, 2*N, 2*N))
    
    # build hamiltonian, i and k are uparrow, <ij|kl>
    # a b a b
    for i in range(2*N):
        for j in range(N,2*N):
            for k in range(2*N):
                for l in range(N,2*N):
                    delta_ij = 1 if i == j+N or i+N == j else 0
                    delta_kl = 1 if k == l+N or k+N == l else 0
                    delta_il = 1 if i == k else 0
                    h2[i, j, k, l] = delta_ij * delta_kl #* (1 - delta_il)
    # b a b a 
    for i in range(N,2*N):
        for j in range(N):
            for k in range(N,2*N):
                for l in range(N):
                    delta_ij = 1 if i == j+N or i+N == j else 0
                    delta_kl = 1 if k == l+N or k+N == l else 0
                    delta_il = 1 if i == k else 0
                    h2[i, j, k, l] = delta_ij * delta_kl #* (1 - delta_il)

    return h2

In [3]:
# make a M x M adjacency matrix 
M = 2
adjacency = np.ones((M,M))
print(adjacency)

[[1. 1.]
 [1. 1.]]


In [4]:
#HamRG(mu: ndarray, J_eq: ndarray, connectivity: ndarray | None = None)
RG = moha.HamRG(0, 1.0, adjacency)

In [5]:
h2_RG = RG.generate_two_body_integral(dense=True,  basis='spinorbital basis' ,sym=8)

how many are non 0 for 2 spatial orbitals, i.e. $\{\bra{i}, \bra{j}, \bra{\bar i}, \bra{\bar j} \}$, Excatly one, because every member needs to have his buddy and there are only two members. Since these are spinorbitals every member can also only be occupied once.  

The number of twinning pairs willl be $M$ since each spatial basis set entry has one twin. The number of non zero elements in H2 will be $M(M-1)/2$, which is the number of pairs that can be formed from the twinning pairs.

In [6]:
# predict properties of the seniority zero h2
print(f"number of twin pairs {M}")
B=M*(M-1)//2
print(f"number of pairs of pairs {B}")

number of twin pairs 2
number of pairs of pairs 1


In [7]:
np.where(h2_RG != 0)

(array([0, 0, 1, 1, 2, 2, 3, 3]),
 array([2, 3, 2, 3, 0, 1, 0, 1]),
 array([1, 1, 0, 0, 3, 3, 2, 2]),
 array([3, 2, 3, 2, 1, 0, 1, 0]))

In [8]:
h2_own = RanGo(M)

In [9]:
np.where(h2_own != 0)

(array([0, 0, 1, 1, 2, 2, 3, 3]),
 array([2, 2, 3, 3, 0, 0, 1, 1]),
 array([0, 1, 0, 1, 2, 3, 2, 3]),
 array([2, 3, 2, 3, 0, 1, 0, 1]))